<a href="https://colab.research.google.com/github/chacha86/bigData1/blob/main/merge_%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import random as rd

# 데스트 데이터

# 이름 국어 영어 수학 100개 정도 테스트 데이터
mid = ['철', '민', '지', '현', '혜', '준', '명']
last = ['수', '연', '혜', '진', '운', '동', '희']

names = []
rd.seed(100) # 랜덤값 고정을 위해 시드 설정

# mid와 last에서 문자를 무작위로 뽑아 100개의 랜덤한 이름 생성
for i in range(100) :
  mid_idx = rd.randint(0, 6)
  last_idx = rd.randint(0, 6)

  name = mid[mid_idx] + last[last_idx]

  names.append(name)

# numpy를 이용해 국영수 세 과목의 점수를 랜덤하게 100개씩 생성
np.random.seed(100)
korean = np.random.randint(60, 100, 100)
english = np.random.randint(60, 100, 100)
math = np.random.randint(60, 100, 100)
nums = np.arange(1, 101)

# 딕셔너리로 조합
data = {
    "번호" : nums,
    "이름" : names,
    "국어" : korean,
    "영어" : english,
    "수학" : math
}
data

# 데이터프레임으로 생성
df = pd.DataFrame(data)

# 그룹핑을 하기 위해 지역 데이터 생성
locations = ['서울', '대전', '대구', '부산', '울산', '광주', '인천']
np.random.seed(10)
arr = np.random.randint(0, 6, 100)

locations = np.array(locations)

# 데이터프레임에 추가
df['지역명'] = locations[arr]

sums = df.loc[:, '국어' : '수학'].sum(axis=1)
avgs = df.loc[:, '국어' : '수학'].mean(axis=1)

df = df.assign(총점=sums, 평균=avgs)

In [2]:
# 학생 국영수 점수
df1 = pd.DataFrame(data)
df2 = None
df3 = None
# 학생 신상 정보
# 번호, 이름, 지역, 성별
# 1, 0 - 0. 남자, 1. 여자
sexes = np.random.randint(0, 2, 100)

def init_df2() :
  global df2
  locations = ['서울', '대전', '대구', '부산', '울산', '광주', '인천']
  np.random.seed(10)
  arr = np.random.randint(0, 6, 100)
  
  locations = np.array(locations)
  locations = locations[arr]
  
  
  data2 = {
      "번호" : nums[:50],
      "이름" : names[:50],
      "지역" : locations[:50],
      "성별" : sexes[:50]
  }
  
  df2 = pd.DataFrame(data2)

  man_idx = df2['성별'] == 0
  df2.loc[man_idx, '성별'] = 'M'
  woman_idx = df2['성별'] == 1
  df2.loc[woman_idx, '성별'] = 'F'
  

def init_df3() :
  global df3
  locations2 = ['daejon', 'daeku', 'seoul', 'pusan', 'incheon', 'ulsan', 'kwangju', 'DAEJON', 'Seoul', 'Busan', 'SEOUL']
  
  np.random.seed(10)
  arr = np.random.randint(0, 11, 50)
  
  locations = np.array(locations2)
  locations2 = locations[arr]
  
  data3 = {
      "No" : nums[50:],
      "Name" : names[50:],
      "Loc" : locations2,
      "Sex" : sexes[50:]
  }
  
  df3 = pd.DataFrame(data3)
  df3['No'] = df3['No'].astype('str') + '번'
  
  m_idx = df3['Sex'] == 0
  df3.loc[m_idx, 'Sex'] = '남자'
  w_idx = df3['Sex'] == 1
  df3.loc[w_idx, 'Sex'] = '여자'

In [8]:
init_df2()
init_df3()
# 학생들의 국영수
df1

# 학생 신상정보1
df2

# 학생 신상정보2
df3

# 학생들의 국영수 점수랑 신상정보를 같이 보고싶다.

# 1. 신상정보2의 번호 데이터에 번이라는 문자가 들어가 있다. 번이라는 글자를 없애고 숫자로 바꾸어 신상정보1과 데이터 타입을 맞춘다.
df3['No'] = df3['No'].str.replace('번', '').astype('int64')

# 2. 신상정보1과 신상정보2의 데이터 표현이 다르다. 데이터를 맞춰주자.
# 먼저 신상정보2의 영어로 된 데이터를 한글로 바꿔야 한다. 
# 중복되는 값을 소문자 처리, 문자 치환을 이용해서 처리한 후 각 영문값을 한글로 바꿔줌.

df3['Loc'].unique() # unique() 함수를 이용하면 중복 없이 데이터 현황을 파악할 수 

df3['Loc'] = df3['Loc'].str.lower()
df3['Loc'].unique()

df3['Loc'] = df3['Loc'].replace('pusan', 'busan')

city_list_eng = df3['Loc'].unique()

city_list_kor = ['부산', '인천', '대전', '대구', '서울', '광주', '울산']

df3['Loc'] = df3['Loc'].replace(city_list_eng, city_list_kor)

df2['성별'] = df2['성별'].replace(['M', 'F'], ['남자', '여자'])
df2['번호']

# 3.신상정보1의 컬럼명에 맞추어 신상정보2의 컬럼명 수정
df3.columns = ['번호', '이름', '지역', '성별']
df3 = pd.concat((df2, df3), axis=0, ignore_index=True)
                
# 4.완성된 학생신상정보를 학생 성적정보에 붙이기
rst = pd.concat((df1, df3[['지역', '성별']]), axis=1)

rst


,번호,이름,국어,영어,수학,지역,성별
0,1,민진,68,77,81,대전,남자
1,2,현희,84,97,88,광주,여자
2,3,민동,63,85,64,울산,여자
3,4,현동,99,95,79,서울,여자
4,5,지진,83,76,67,대전,남자
...,...,...,...,...,...,...,...
95,96,준진,99,63,65,대전,여자
96,97,철운,90,86,62,서울,여자
97,98,현진,90,96,60,대구,남자
98,99,현혜,62,69,69,대전,여자


In [9]:
#merge 연습

# 선택과목 테스트 데이터 만들기

subjects = ['사회문화', '경제', '세계사', '한국사', '지리', '윤리', '정치', '물리', '지구과학', '생물', '화학', '기타'] # 선택과목명
teachers = ['김문화', '오시장', '박시저', '이세종', '이단구', '박헤겔', '나링컨', '이타인', '김지질', '장세포', '주기율', '김기타'] # 과목담당 선생님
subject_nums = np.arange(1, 13) # 과목 번호
data6 = {
    '과목번호' : subject_nums,
    '과목명' : subjects,
    '담당선생님' : teachers 
}

df6 = pd.DataFrame(data6) # 선택 과목 데이터프레임

np.random.seed(10) # 시드 고정
sub_cnts = np.random.randint(0, 5, 100) # 각 학생 별 선택 과목 개수 (100 명중 한 학생당 0 ~ 4개를 선택한다는 의미)

df7 = pd.DataFrame() # 비어있는 데이터 프레임(학생의 선택과목 정보를 저장하기 위함)

idx = 0 # 각 학생을 순서
for cnt in sub_cnts : # 학생별 선택 과목 개수를 하나씩 꺼내어
  nums = np.random.choice(11, cnt, replace=False) # 0 ~ 11 사이의 중복되지 않는 랜덤 정수를 cnt만큼 뽑음.(여기서 0 ~ 11 사이의 값은 선택과목을 의미함)
  df6.loc[nums, '학번'] = idx + 1 # nums에 저장된 랜덤한 선택과목 인덱스를 이용해 df6(선택과목 데이터프레임)에서 특정 로우를 뽑아 학생번호를 부여해줌.
  df7 = pd.concat((df7, df6.iloc[nums])) # 비어있는 df7에 만들어진 학생들의 선택과목을 아래로 누적해서 붙여줌
  idx += 1 # 다음 학생으로 넘어감

# 중요한 컬럼으로 재배치
rst2 = df7[['학번', '과목번호', '과목명', '담당선생님']]

# 학생정보(rst)와 선택과목 정보(rst2)
# rst3 = pd.merge(rst, rst2) # 같은 키가 존재하지 않아 비교 불가

# 방법1 - 컬럼명을 일치시켜줌
# rst2.columns.values[0] = '번호'
# rst2 = rst2.rename(columns={'학번' : '번호'})
# rst3 = pd.merge(rst, rst2) # 같은 키가 존재하지 않아 비교 불가

# 방법2 - left_on, right_on 옵션을 사용
# rst3 = pd.merge(rst, rst2, left_on='번호', right_on='학번') # left_on, right_on 은 각 데이터프레임의 키를 직접 지정해주는 방식. 왼쪽 df의 '번호' 컬럼과 오른쪽 df의 '학번' 컬럼의 값이 같은 것만 붙여준다.


# merge 방법 - how라는 인수를 설정해주어 처리.
# 1. inner
#  - 키로 지정한 컬럼이 일치하는 행만 붙여서 결과로 보여줌(교집합). 일치하지 않는 행은 결과에서 제외
# 2. left
#  - inner의 결과에서 제외된 행중 왼쪽 df(데이터프레임)의 행 기준으로 오른쪽의 값을 모두 NaN처리해서 보여줌
# 3. right
#  - inner의 결과에서 제외된 행중 오른쪽 df(데이터프레임)의 행 기준으로 왼쪽의 값을 모두 NaN처리해서 보여줌
# 4. outer
#  - left와 right의 결과를 합해서 보여줌

# 아래는 예시.

data1 = {
    'col1' : [1,2,3,4],
    'col2' : ['a','b','c','d'],
    'col3' : ['e','f','g','h']
}

data2 = {
    'col4' : [1,3,5,7],
    'col5' : ['i','j','k','l'],
    'col6' : ['m','n','o','p']
}

tdf1 = pd.DataFrame(data1)
tdf1

tdf2 = pd.DataFrame(data2)
tdf2

print(tdf1)
print(tdf2)

# 해당 테스트에서는 key로 col1과 col4를 잡는다. (왼쪽의 col1과 오른쪽의 col4의 값을 가지고 붙인다는 의미)
# inner - 양쪽 모두 존재하는 1, 3만 결과로 잡힘
pd.merge(tdf1, tdf2, left_on='col1', right_on='col4', how='inner') # how를 생략해도 default inner 이다.

# left - 왼쪽에 존재하는 모든 값 (1,2,3,4)이 결과로 잡힘. 오른쪽에 없는 2, 4의 경우 NaN처리 되어 붙는다.
pd.merge(tdf1, tdf2, left_on='col1', right_on='col4', how='left') 

# right - 오른쪽에 존재하는 모든 값 (1,3,5,7)이 결과로 잡힘. 왼쪽에 없는 5, 7의 경우 NaN처리 되어 붙는다.
pd.merge(tdf1, tdf2, left_on='col1', right_on='col4', how='right') 

# outer - 양쪽에 존재하는 모든 값 (1,2,3,4,5,7)이 결과로 잡힘. 없는 쪽은 NaN처리 되어 붙는다.
pd.merge(tdf1, tdf2, left_on='col1', right_on='col4', how='outer') 

   col1 col2 col3
0     1    a    e
1     2    b    f
2     3    c    g
3     4    d    h
   col4 col5 col6
0     1    i    m
1     3    j    n
2     5    k    o
3     7    l    p


,col1,col2,col3,col4,col5,col6
0,1.0,a,e,1.0,i,m
1,2.0,b,f,NaN,NaN,NaN
2,3.0,c,g,3.0,j,n
3,4.0,d,h,NaN,NaN,NaN
4,NaN,NaN,NaN,5.0,k,o
5,NaN,NaN,NaN,7.0,l,p
